In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
filename = "mistral-7b-instruct-v0.2.Q8_0.gguf"

tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename, device_map="auto" )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Merges were not in checkpoint, building merges on the fly.


  0%|          | 0/32000 [00:00<?, ?it/s]

Converting and de-quantizing GGUF tensors...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 291/291 [00:47<00:00,  6.18it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [6]:
!pip show gguf

Name: gguf
Version: 0.14.0
Summary: Read and write ML models in GGUF for GGML
Home-page: https://ggml.ai
Author: GGML
Author-email: ggml@ggml.ai
License: 
Location: C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages
Requires: numpy, pyyaml, sentencepiece, tqdm
Required-by: 


In [4]:
!pip show torch

Name: torch
Version: 2.5.1+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: sentence-transformers, torchaudio, torchvision


In [23]:
from transformers import StoppingCriteria
class MyStoppingCriteria(StoppingCriteria):
    def __init__(self, target_sequence, prompt):
        self.target_sequence = target_sequence
        self.prompt=prompt
    
    def __call__(self, input_ids, scores, **kwargs):
        # Get the generated text as a string
        generated_text = tokenizer.decode(input_ids[0])
        generated_text = generated_text.replace(self.prompt,'')
        # Check if the target sequence appears in the generated text
        if self.target_sequence in generated_text:
            return True  # Stop generation
    
        return False  # Continue generation
    
    def __len__(self):
        return 1
    
    def __iter__(self):
        yield self

In [28]:
prompt = """
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hatespeech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

User: fuck that nigger
Assistant:"""

In [29]:
%%time
inputs = tokenizer([prompt], return_tensors="pt").to('cuda')
outputs = model.generate(
    **inputs,
    max_new_tokens=20,
    # num_beams=1,
    num_return_sequences=1,
    return_dict_in_generate=True,
    output_scores=True,
    stopping_criteria=MyStoppingCriteria("User:", prompt)
)

# Decode the generated sequences
generated_texts = [tokenizer.decode(seq, skip_special_tokens=True) for seq in outputs.sequences]

# Print the generated texts
for i, text in enumerate(generated_texts, 1):
    print(f"Output {i}: {text}")

Output 1: 
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hatespeech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

User: i love you
Assistant: normal

User:
CPU times: total: 3.28 s
Wall time: 3.36 s


In [2]:
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.2",
  "prompt": "Why is the sky blue?"
}'

IndentationError: unexpected indent (2385289802.py, line 2)

In [4]:
import requests
import json

In [5]:
# Define LLM inference function to use later
API_URL = "http://localhost:11434/api/chat"

def llm(model_name, system_prompt, input_query):
    # Construct the request payload
    payload = {
        "messages": [
            {"content": system_prompt, "role": "system"},
            {"content": input_query, "role": "user"}
        ],
        "model": model_name,
        "options": {}
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    # Send the POST request
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    
    # Process the response
    if response.status_code == 200:
        print("Response received successfully:")
        response = response.json() # json.dumps(, indent=4)
    else:
        print(f"Failed to retrieve response. Status code: {response.status_code}")

    return response

In [6]:
# Testing LLM
system_prompt = """
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hatespeech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

Here is user input and reply with only one word"""
input_query = "i hate black men"
model_name = "llama3.3:70B-Instruct-Q2_K"

In [9]:
%%time

response = llm(model_name, system_prompt, input_query)
# explaination = response['response']
print(response)

Response received successfully:


JSONDecodeError: Extra data: line 2 column 1 (char 141)

In [32]:
# import requests
# import json

# # Define LLM inference function to use later
# API_URL = "http://localhost:11434/api/generate"

# def llm(model_name, system_prompt, input_query):
#     # Construct the request payload
#     payload = {
#         "model": model_name,
#         "prompt": system_prompt + "\n" + input_query,  # Concatenating system and user prompts
#         "stream": False  # Use non-streaming response for simplicity
#     }
    
#     # Set the request headers
#     headers = {
#         "accept": "application/json",
#         "Content-Type": "application/json"
#     }
    
#     # Send the POST request
#     response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    
#     # Process the response
#     if response.status_code == 200:
#         print("Response received successfully:")
#         response_data = response.json()
#         print(json.dumps(response_data, indent=4))  # Pretty-print the JSON response
#         return response_data
#     else:
#         print(f"Failed to retrieve response. Status code: {response.status_code}")
#         print("Response content:", response.text)
#         return None

# # Example usage
# model_name = "mistral:7b-instruct-v0.2-q8_0"
# system_prompt = "You are a helpful assistant."
# input_query = "Why is the sky blue?"
# response = llm(model_name, system_prompt, input_query)


In [1]:
import os
import pandas as pd
os.chdir('..')

In [2]:
df = pd.read_csv("Data/dataset.csv", index_col=0)

In [3]:
df.head()

,post_id,tweet_text,key_features,target,label,annotator_1_label,annotator_1_target,annotator_2_label,annotator_2_target,annotator_3_label,annotator_3_target
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[],NaN,normal,normal,NaN,normal,NaN,normal,NaN
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,[],NaN,normal,normal,NaN,normal,NaN,normal,NaN
2,1178793830532956161_twitter,nawt yall niggers ignoring me,[],"African, None",normal,normal,African,normal,NaN,hatespeech,African
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,"['ching', 'chong']",Asian,hatespeech,hatespeech,Asian,offensive,Asian,hatespeech,Asian
4,1179085312976445440_twitter,this bitch in whataburger eating a burger with...,"['bitch', 'i', 'hate', 'white', 'bitches']","Caucasian, Women",hatespeech,hatespeech,"Caucasian, Women",hatespeech,"Women, Caucasian",offensive,"Women, Caucasian"


In [6]:
import requests
import json

# Define LLM inference function to use later
API_URL = "http://localhost:11434/api/chat"

def llm(model_name, system_prompt, input_query, stream=False):
    # Construct the request payload
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_query}
        ],
        "stream": stream  # Ensure streaming behavior matches API expectations
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    try:
        # Send the POST request
        response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
        
        # Check for errors
        if response.status_code == 200:
            # print("Response received successfully:")
            return response.json()  # Parse JSON response
        else:
            print(f"Failed to retrieve response. Status code: {response.status_code}, Details: {response.text}")
            return None
    except requests.RequestException as e:
        print(f"An error occurred while connecting to the API: {e}")
        return None

# Example usage
##response = llm("llama3.3:70B-Instruct-Q2_K", "You are a helpful assistant.", "Why is the sky blue?")
# print(response)


In [26]:
# Testing LLM
system_prompt = """
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hate speech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

Here is user input and reply with only one word ONLY"""
input_query = "oh my, i think you should do some work on your skin tone"
model_name = "llama3.3:70b-instruct-q5_K_M"

In [28]:
%%time

response = llm(model_name, system_prompt, input_query)
# explaination = response['response']
print(response['message']['content'].strip())

offensive
CPU times: total: 15.6 ms
Wall time: 4.29 s


In [ ]:
from tqdm import tqdm

# Initialize tqdm for pandas
tqdm.pandas()

df["mistral"] = df["tweet_text"].progress_apply(lambda text: llm(model_name, system_prompt, text))

In [17]:
from tqdm import tqdm
mistral_results = []
# Loop through the DataFrame with a progress bar
for text in tqdm(df['tweet_text'], desc="Processing tweets"):
    # Strip spaces from input
    input_query = text
    # Get the LLM response
    response = llm(model_name, system_prompt, input_query)
    # Extract and strip the response content
    processed_content = response["message"]["content"].strip()
    # Append the processed content to the results list
    mistral_results.append(processed_content)

# Add the results to a new column in the DataFrame
df["mistral"] = mistral_results

Processing tweets: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20148/20148 [13:52:33<00:00,  2.48s/it]


In [35]:
df.to_csv("dataset_classification_with_mistral")

In [21]:
# Assign true labels and predicted labels
y_test = df["label"]
y_pred = df["mistral"]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))  #

Accuracy: 0.2724

Classification Report:


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
from sklearn.metrics import accuracy_score, classification_report

In [23]:
df["mistral"].value_counts()

mistral
offensive                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             7895
normal                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [28]:
df["mistral_cleaned"] = df["mistral"].str.split('[.\n(]', n=1).str[0].str.strip()

In [29]:
df['label'].value_counts()

label
normal        8153
hatespeech    6234
offensive     5761
Name: count, dtype: int64

In [30]:
df["mistral_cleaned"].value_counts().head(50)

mistral_cleaned
offensive                                                                                                                                                                                                        14285
normal                                                                                                                                                                                                            4155
hate speech                                                                                                                                                                                                       1431
Hatespeech                                                                                                                                                                                                         122
hatespeech                                                                                                                  

In [33]:
normalization_map = {
    "offensive": "offensive",
    "Offensive": "offensive",
    "hate speech": "hatespeech",
    "Hatespeech": "hatespeech",
    "hatespeech": "hatespeech",
    "hatedspeech": "hatespeech",
    "haterspeech": "hatespeech",
    "normal": "normal",
    "Normal": "normal"
}
df["mistral_normalized"] = df["mistral_cleaned"].replace(normalization_map)
# Normalize the 'mistral_cleaned' column
# df["mistral_normalized"] = df["mistral_cleaned"].str.lower().map(normalization_map).fillna("hatespeech")

# Display value counts for the normalized column
df["mistral_normalized"].value_counts()

mistral_normalized
offensive                                                                                                                                                                                                        14290
normal                                                                                                                                                                                                            4155
hatespeech                                                                                                                                                                                                        1672
I will do my best to classify the text based on the given instructions                                                                                                                                               3
hatemspeech                                                                                                              

In [32]:
# Assign true labels and predicted labels
y_test = df["label"]
y_pred = df["mistral_normalized"]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))  #

Accuracy: 0.4453

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.75      0.20      0.32      6234
      normal       0.73      0.37      0.49      8153
   offensive       0.33      0.81      0.47      5761

    accuracy                           0.45     20148
   macro avg       0.60      0.46      0.43     20148
weighted avg       0.62      0.45      0.43     20148



In [34]:
# Assign true labels and predicted labels
y_test = df["label"]
y_pred = df["mistral_normalized"]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))  #

Accuracy: 0.4452

Classification Report:
                                                                                                                                                                                                               precision    recall  f1-score   support

                                                                                                                                             I cannot classify the text without knowing what it actually says       0.00      0.00      0.00         0
                                                                                                                                                  I cannot classify the text without knowing what the text is       0.00      0.00      0.00         0
                                                                                                                                                           I cannot classify the text without seeing it first       0.00 

In [54]:
# Define LLM inference function to use later
API_URL = "https://inf.cl.uni-trier.de/chat/"

def llm(model_name, system_prompt, input_query):
    # Construct the request payload
    payload = {
        "messages": [
            {"content": system_prompt, "role": "system"},
            {"content": input_query, "role": "user"}
        ],
        "model": model_name,
        "options": {}
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    # Send the POST request
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    
    # Process the response
    if response.status_code == 200:
        print("Response received successfully:")
        response = response.json() # json.dumps(, indent=4)
    else:
        print(f"Failed to retrieve response. Status code: {response.status_code}")

    return response


In [44]:
# Testing LLM
system_prompt = """
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hate speech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

Here is user input and reply with only one word ONLY"""
input_query = "i hate your black daughter"
model_name = "llama3.3:70B-Instruct-Q2_K"

In [47]:
%%time

response = llm(model_name, system_prompt, input_query)
# explaination = response['response']
# print(response['response'])

CPU times: total: 15.6 ms
Wall time: 2.83 s


## llama

In [10]:
import pandas as pd
import os

# os.chdir('..')
df = pd.read_csv("C:\\MachineLearning\\UniTrier\\RCS\\twon-hate-speech-detection\\Data\\dataset.csv", index_col=0)

In [2]:
import requests
import json

# Define LLM inference function to use later
API_URL = "http://localhost:11434/api/chat"

def llm(model_name, system_prompt, input_query, stream=False):
    # Construct the request payload
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_query}
        ],
        "stream": stream  # Ensure streaming behavior matches API expectations
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    try:
        # Send the POST request
        response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
        
        # Check for errors
        if response.status_code == 200:
            # print("Response received successfully:")
            return response.json()  # Parse JSON response
        else:
            print(f"Failed to retrieve response. Status code: {response.status_code}, Details: {response.text}")
            return None
    except requests.RequestException as e:
        print(f"An error occurred while connecting to the API: {e}")
        return None


In [3]:
# Testing LLM
system_prompt = """
Lets classify the following text for hate speech.

if its hatespeech your reply will be: "hate speech"
if its not hatespeech your reply will be : "normal"
if its offensive and not hatespeech, your reply will be : "offensive"

Here is user input and reply with only one word ONLY"""
input_query = "oh my, i think you should do some work on your skin tone"
model_name = "llama3.3:70b-instruct-q5_K_M"

In [4]:
df = df.head(10000)

In [5]:
df.head()

,post_id,tweet_text,key_features,target,label,annotator_1_label,annotator_1_target,annotator_2_label,annotator_2_target,annotator_3_label,annotator_3_target
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[],NaN,normal,normal,NaN,normal,NaN,normal,NaN
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,[],NaN,normal,normal,NaN,normal,NaN,normal,NaN
2,1178793830532956161_twitter,nawt yall niggers ignoring me,[],"African, None",normal,normal,African,normal,NaN,hatespeech,African
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,"['ching', 'chong']",Asian,hatespeech,hatespeech,Asian,offensive,Asian,hatespeech,Asian
4,1179085312976445440_twitter,this bitch in whataburger eating a burger with...,"['bitch', 'i', 'hate', 'white', 'bitches']","Caucasian, Women",hatespeech,hatespeech,"Caucasian, Women",hatespeech,"Women, Caucasian",offensive,"Women, Caucasian"


In [ ]:
from tqdm import tqdm
llama_results = []
# Loop through the DataFrame with a progress bar
for text in tqdm(df['tweet_text'], desc="Processing tweets"):
    # Strip spaces from input
    input_query = text
    # Get the LLM response
    response = llm(model_name, system_prompt, input_query)
    # Extract and strip the response content
    processed_content = response["message"]["content"].strip()
    # Append the processed content to the results list
    llama_results.append(processed_content)

# Add the results to a new column in the DataFrame
df["llama"] = llama_results

In [ ]:
df.to_csv("Data/dataset_classification_with_llama_Q5.csv")

In [12]:
len(llama_results)

4464

In [20]:
import numpy as np
df["llama_Q5"] = np.nan
df["llama_Q5"] = df["llama_Q5"].astype(object)

In [21]:
df.loc[:len(llama_results) - 1, "llama_Q5"] = llama_results

In [22]:
df["llama_Q5"].value_counts()

llama_Q5
hate speech                                                                                                                                                                                                                                                 2888
offensive                                                                                                                                                                                                                                                    846
normal                                                                                                                                                                                                                                                       724
I cannot classify hate speech. Can I help you with something else?                                                                                                                                                          

In [24]:
df.isna().sum()

post_id                   0
tweet_text                0
key_features              0
target                 3918
label                     0
annotator_1_label         0
annotator_1_target     7127
annotator_2_label         0
annotator_2_target     6997
annotator_3_label         0
annotator_3_target     7068
llama_Q5              15684
dtype: int64

In [26]:
df.to_csv("dataset_classification_with_llama_Q5.csv")